# Análise das notícias sob a ótica ESG: geração da base com sentimentos

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG

In [1]:
import warnings
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import seaborn as sns
import datetime  as dt
from noticias_timeline import plota_timeline
from noticias_processamento_texto import *
from noticias_graficos import *
from analise_sentimento_modelo_gpt import *
from classificador_esg import aplica_classificador_esg
import pickle
from cotacoes import *

warnings.filterwarnings('ignore')
lista_empresas = 'datasets/lista_empresas.xlsx'
base_noticias = 'datasets/base_noticias.xlsx'
base_noticias_saida = 'datasets/sentimento_base_noticias.xlsx'
base_noticias_saida_short = 'datasets/sentimento_base_noticias_short.xlsx'
caminho_cache='datasets/gpt_cache.xlsx'

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Lendo base de notícias

In [2]:
df = pd.read_excel(base_noticias)

df['fonte'] = df['fonte'].apply(trata_nome_fontes)
df['data_publicacao'] = df['data_publicacao'].dt.date
df = df.sort_values(by=['empresa', 'data_publicacao'])
df['titulo_par'] = df['titulo'].str.strip().apply(remove_acentos).str[:15]
df = df.drop_duplicates(subset=['empresa', 'fonte', 'data_publicacao', 'titulo_par'], keep='first')
del df['titulo_par']

# ajustes
df = ajusta_nomes_empresas_dataframe(df)

# cria hash
df['hash'] = df.apply(lambda row: criar_hash_noticia(row['texto_completo'], row['empresa'], titulo=row['titulo'], data=row['data_publicacao']), axis=1)

# le lista empresas
df_empresas = pd.read_excel(lista_empresas)

## Aplicação Classificador SVM

In [3]:
# utiliza o classificador por ML para determinar o assunto da noticia: E, S, G ou outros
vect = pickle.load(open('models/svm_vetorizador.sav', 'rb'))
model = pickle.load(open('models/svm_classificador.sav', 'rb'))

if len(df.loc[ (pd.isnull(df['classificacao_ml']) &   (~pd.isnull(df['texto_completo']))), 'classificacao_ml']) >0:
    df.loc[ (pd.isnull(df['classificacao_ml']) &   (~pd.isnull(df['texto_completo']))), 'classificacao_ml'] = aplica_classificador_esg(vect, model, 
                                                                                                              df.loc[ (pd.isnull(df['classificacao_ml']) &   (~pd.isnull(df['texto_completo'])))], 
                                                                                                              comparar_com_real=False, col_texto_origem='texto_completo', 
                                                                                                              col_texto_saida='texto_ajustado', col_classe_verdadeira='classificacao')       
    
# salva na base original para cache
df.to_excel(base_noticias, index=False)

## Aplicação do CHAT GPT

In [ ]:
df = pd.read_excel(base_noticias)

# abre o cache
df_cache = pd.read_excel(caminho_cache)
dic_cache = df_cache.set_index('hash')['resposta'].to_dict()
tam_inicial = len(dic_cache)


try:
    df['gpt_resposta_completa'] = df.apply(lambda row: classifica_sentimento_noticia_gpt(row['data_publicacao'], row['titulo'], row['texto_completo'], row['empresa'], dicionario_cache=dic_cache)  if (  (not pd.isnull(row['texto_completo'])) and row['classificacao_ml'] != 'Outros' and ( pd.isnull(row['gpt_resposta_completa'])) ) else row['gpt_resposta_completa'], axis=1)
finally:                                                                    
    #salva o cache do gpt
    tam_final = len(dic_cache)
    print('Realizadas ' + str(tam_final - tam_inicial) + ' chamadas à API do GPT.')
    df_cache = pd.DataFrame({'hash': dic_cache.keys(),  'resposta': dic_cache.values()})
    df_cache.to_excel(caminho_cache, index=False)

# faz os filtros
df.to_excel(base_noticias, index=False)
df = df[~pd.isnull(df['texto_completo'])]
df = df[df['classificacao_ml'] != 'Outros']  

# tratando
df = gera_colunas_gpt(df, coluna_resposta_gpt='gpt_resposta_completa')
df = filtros_pos_gpt(df)
df['polaridade'] = df['gpt_polaridade']  # substitui metodo anterior
# resumos vazios
df['gpt_resumo'] = df.apply(lambda row: row['titulo'] if pd.isnull(row['gpt_resumo']) else row['gpt_resumo'] , axis=1)

df['gpt_lista_respostas'] = df['gpt_lista_respostas'].astype(str)  # para permitir remover duplicatas
df = filtrar_noticias_pos_coleta_modelo_simplificado(df, df_empresas)

df.to_excel(base_noticias_saida, index=False)

# versao reduzida para o site
df_short = df.loc[: , ['hash', 'titulo', 'data_publicacao', 'url', 'fonte', 'empresa', 'Nome',  'CNPJ', 'Razão social', 'Setor', 'Código',  'classificacao', 'gpt_resumo', 'polaridade']]
df_short.to_excel(base_noticias_saida_short, index=False)

print('Processamento concluído')

Realizadas 0 chamadas à API do GPT.
